In [5]:
from os import makedirs
from os import listdir
from numpy import asarray
from numpy import save
from matplotlib import pyplot
from keras.applications.vgg16 import VGG16
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from shutil import copyfile
from random import seed
from random import random
import sys

In [6]:
dataset_home = 'Dataset_cells/'
subdirs = ['train/', 'test/']
for subdir in subdirs:
    # create label subdirectories
    labeldirs = ['HealthyRBC/', 'Sickles/']
    for labldir in labeldirs:
        newdir = dataset_home + subdir + labldir
        makedirs(newdir, exist_ok=True)
        
# seed random number generator
seed(1)
# define ratio of pictures to use for validation
val_ratio = 0.25
# copy training dataset images into subdirectories
src_directory = 'Dataset/'

for file in listdir(src_directory):
    src = src_directory + '/' + file
    dst_dir = 'train/'
    if random() < val_ratio:
        dst_dir = 'test/'
    if file.startswith('HealthyRBC'):
        dst = dataset_home + dst_dir + 'HealthyRBC/'  + file
        copyfile(src, dst)
    elif file.startswith('Sickle'):
        dst = dataset_home + dst_dir + 'Sickles/'  + file
        copyfile(src, dst)

In [7]:
 
# define cnn model
def define_model():
    # load model
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()
 
# run the test harness for evaluating a model
def run_test_harness():
    # define model
    model = define_model()
    # create data generator
    datagen = ImageDataGenerator(featurewise_center=True)
    # specify imagenet mean values for centering
    datagen.mean = [123.68, 116.779, 103.939]
    # prepare iterator
    train_it = datagen.flow_from_directory('Dataset_cells/train/',
        class_mode='binary', batch_size=64, target_size=(224, 224))
    test_it = datagen.flow_from_directory('Dataset_cells/test/',
        class_mode='binary', batch_size=64, target_size=(224, 224))
    # fit model
    history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
        validation_data=test_it, validation_steps=len(test_it), epochs=10, verbose=1)
    # evaluate model
    _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
    print('> %.3f' % (acc * 100.0))
    # learning curves
    summarize_diagnostics(history)
    model.save('final_model.h5')

 
# entry point, run the test harness
run_test_harness()

Found 127 images belonging to 2 classes.
Found 48 images belonging to 2 classes.
Epoch 1/10
2/2 [==============================] - 6s 3s/step - loss: 7.4298 - accuracy: 0.4646 - val_loss: 2.8683 - val_accuracy: 0.5208
Epoch 2/10
2/2 [==============================] - 1s 311ms/step - loss: 1.2220 - accuracy: 0.7717 - val_loss: 4.4310 - val_accuracy: 0.5208
Epoch 3/10
2/2 [==============================] - 3s 2s/step - loss: 2.4867 - accuracy: 0.7165 - val_loss: 7.4778e-08 - val_accuracy: 1.0000
Epoch 4/10
2/2 [==============================] - 3s 2s/step - loss: 8.6716e-06 - accuracy: 1.0000 - val_loss: 5.2592e-06 - val_accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 3s 2s/step - loss: 0.0211 - accuracy: 0.9843 - val_loss: 3.9557e-04 - val_accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 3s 2s/step - loss: 0.0920 - accuracy: 0.9685 - val_loss: 2.4396e-04 - val_accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 3s 2s/step - loss: 0.034

In [9]:
# load and prepare the image
def load_image(filename):
    # load the image
    img = load_img(filename, target_size=(224, 224))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 3 channels
    img = img.reshape(1, 224, 224, 3)
    # center pixel data
    img = img.astype('float32')
    img = img - [123.68, 116.779, 103.939]
    return img

# load an image and predict the class
def run_example(filename):
    # load the image
    img = load_image(filename)
    # load model
    model = load_model('final_model.h5')
    # predict the class
    result = model.predict(img)
    if int(result[0]) < 1:
        print('Image: ' + filename + ' is a Healthy RBC Image' ) 
    else:
        print('Image: ' + filename + ' is a Sickle Cells Image')

# copy training dataset images into subdirectories
directory = 'Images_For_Test'
for filename in listdir(directory):
    if filename.endswith(".jpg"):
        run_example(directory + '/' +filename)
        

Image: Images_For_Test/HealthyRBC104.jpg is a Healthy RBC Image
Image: Images_For_Test/HealthyRBC107.jpg is a Healthy RBC Image
Image: Images_For_Test/HealthyRBC108.jpg is a Healthy RBC Image
Image: Images_For_Test/HealthyRBC109.jpg is a Healthy RBC Image
Image: Images_For_Test/HealthyRBC77.jpg is a Healthy RBC Image
Image: Images_For_Test/HealthyRBC78.jpg is a Healthy RBC Image
Image: Images_For_Test/HealthyRBC79.jpg is a Healthy RBC Image
Image: Images_For_Test/HealthyRBC80.jpg is a Healthy RBC Image
Image: Images_For_Test/HealthyRBC90.jpg is a Healthy RBC Image
Image: Images_For_Test/HealthyRBC91.jpg is a Healthy RBC Image
Image: Images_For_Test/HealthyRBC95.jpg is a Healthy RBC Image
Image: Images_For_Test/HealthyRBC96.jpg is a Healthy RBC Image
Image: Images_For_Test/SickleCell103.jpg is a Sickle Cells Image
Image: Images_For_Test/SickleCell115.jpg is a Sickle Cells Image
Image: Images_For_Test/SickleCell117.jpg is a Sickle Cells Image
Image: Images_For_Test/SickleCell118.jpg is a